# Pharoah

Building the structure of the sequence data and preparing it for the RNN is a whole entire job.

In [33]:
import pandas as pd
import numpy as np

admission_table = {'Patient 1': {'PatientID':'A1234-B456', 
                          'Admission ID':[12,34,15], 
                          'AdmissionStartDate':['2019-01-03 9:34:55','2019-02-03 10:50:55','2019-04-03 12:34:55'],
                          'AdmissionEndDate':['2019-01-07 8:45:43','2019-03-04 1:50:32','2019-04-03 5:38:18']},
                   'Patient 2': {'PatientID':'B1234-C456', 
                          'Admission ID':[13,34], 
                          'AdmissionStartDate':['2018-01-03 9:34:55','2018-02-03 10:50:55'],
                          'AdmissionEndDate':['2018-01-07 8:45:43','2018-03-04 1:50:32']}}

admission_table = (pd.concat({k: pd.DataFrame(v) for k, v in admission_table.items()}).reset_index(level=1, drop=True))
admission_table = admission_table.reset_index(drop=True)

In [34]:
admission_table

,PatientID,Admission ID,AdmissionStartDate,AdmissionEndDate
0,A1234-B456,12,2019-01-03 9:34:55,2019-01-07 8:45:43
1,A1234-B456,34,2019-02-03 10:50:55,2019-03-04 1:50:32
2,A1234-B456,15,2019-04-03 12:34:55,2019-04-03 5:38:18
3,B1234-C456,13,2018-01-03 9:34:55,2018-01-07 8:45:43
4,B1234-C456,34,2018-02-03 10:50:55,2018-03-04 1:50:32


In [35]:
Patient_1 = {'PatientID':'A1234-B456', 
             'Admission ID':[12,34,15], 
             'PrimaryDiagnosisCode':[['E11.64','I25.812','I25.10'],
                                     ['E11.64','I25.812','I25.10','780.96','784.0'],
                                     ['E11.64','I25.812','I25.10','786.50','401.9','789.00']],
             'CodingSystem':['ICD-9','ICD-9','ICD-9'],
             'DiagnosisCodeDescription':[['Type 2 diabetes mellitus with hypoglycemia',
                                          'Atherosclerosis of bypass graft of coronary artery of transplanted heart without angina pectoris',
                                          'Atherosclerotic heart disease of native coronary artery without angina pectoris'],
                                         ['Type 2 diabetes mellitus with hypoglycemia',
                                          'Atherosclerosis of bypass graft of coronary artery of transplanted heart without angina pectoris',
                                          'Atherosclerotic heart disease of native coronary artery without angina pectoris',
                                          'Generalized Pain', 'Dizziness and giddiness'],
                                         ['Type 2 diabetes mellitus with hypoglycemia',
                                          'Atherosclerosis of bypass graft of coronary artery of transplanted heart without angina pectoris',
                                          'Atherosclerotic heart disease of native coronary artery without angina pectoris',
                                          'Chest pain, unspecified','Essential hypertension, unspecified',
                                          'Abdominal pain, unspecified site']]}
Patient_2 = {'PatientID':'B1234-C456', 
              'Admission ID':[13,34], 
              'PrimaryDiagnosisCode':[['M05.59','Z13.85','O99.35'],['M05.59','Z13.85','O99.35','D37.0']],
              'CodingSystem':['ICD-9','ICD-9'],
              'DiagnosisCodeDescription':[['Rheumatoid polyneuropathy with rheumatoid arthritis of multiple sites',
                                           'Encounter for screening for nervous system disorders',
                                           'Diseases of the nervous system complicating pregnancy, childbirth, and the puerperium'],
                                          ['Rheumatoid polyneuropathy with rheumatoid arthritis of multiple sites',
                                           'Encounter for screening for nervous system disorders',
                                           'Diseases of the nervous system complicating pregnancy, childbirth, and the puerperium',
                                           'Neoplasm of uncertain behavior of lip, oral cavity and pharynx']]}

In [36]:
def process_ehr(Patient1,Patient2):
    pt_diagnosis_table = [Patient1,Patient2]
    pt_diagnosis_table = pd.concat([pd.DataFrame({k:v for k,v in d.items()}) for d in pt_diagnosis_table])
    
    pt_diagnosis_table = (pt_diagnosis_table.set_index(['PatientID', 'Admission ID','CodingSystem'])
              .apply(lambda x: x.apply(pd.Series).stack())
              .reset_index()
              .drop('level_3', 1))
    return pt_diagnosis_table

def hash_key(df):
    df['HashKey'] = df['PatientID'].\
    apply(lambda x: x.split('-')[0]) + '-' + df['Admission ID'].astype('str')
    cols = [df.columns[-1]] + [col for col in df if col != df.columns[-1]]
    return df[cols]

diagnosis_table = process_ehr(Patient_1,Patient_2)
diagnosis_table.head()

,PatientID,Admission ID,CodingSystem,PrimaryDiagnosisCode,DiagnosisCodeDescription
0,A1234-B456,12,ICD-9,E11.64,Type 2 diabetes mellitus with hypoglycemia
1,A1234-B456,12,ICD-9,I25.812,Atherosclerosis of bypass graft of coronary ar...
2,A1234-B456,12,ICD-9,I25.10,Atherosclerotic heart disease of native corona...
3,A1234-B456,34,ICD-9,E11.64,Type 2 diabetes mellitus with hypoglycemia
4,A1234-B456,34,ICD-9,I25.812,Atherosclerosis of bypass graft of coronary ar...


In [37]:
admission_table = hash_key(admission_table)
diagnosis_table = hash_key(diagnosis_table)

# Write files to data directory
diagnosis_table.to_csv('data/Diagnosis_Table.csv',encoding='UTF-8',index=False)
admission_table.to_csv('data/Admissions_Table.csv',encoding='UTF-8',index=False,date_format='%Y-%m-%d')

In [40]:
# %debug
import pandas as pd
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import warnings
from datetime import datetime
import torch
import pickle
from collections import defaultdict
warnings.filterwarnings('ignore')
sns.set(style='white')

print('Creating visit date mapping')
patHashMap = dict(defaultdict(list))  # this creates a dictionary with a list of values for each patient:[number of visists]
visitMap = dict(defaultdict()) # this creates a dictionary with a mapping of the patientID : visitdates

data = open('data/Admissions_Table.csv','r')
data.readline()[1:] # read every line except the file header

for line in data:
    feature = line.strip().split(',') # split line on , and isolate columns
    print(feature)
    visitDateID = datetime.strptime(feature[3],'%Y-%m-%d %H:%M:%S') 
    patHashMap.setdefault(feature[1], []).append(feature[0]) # create a mapping for each visit for a specific PatientID
    visitMap.setdefault(feature[0], []).append(visitDateID) # create a mapping for each visit for a specific Admission Date

Creating visit date mapping
['A1234-12', 'A1234-B456', '12', '2019-01-03 9:34:55', '2019-01-07 8:45:43']
['A1234-34', 'A1234-B456', '34', '2019-02-03 10:50:55', '2019-03-04 1:50:32']
['A1234-15', 'A1234-B456', '15', '2019-04-03 12:34:55', '2019-04-03 5:38:18']
['B1234-13', 'B1234-C456', '13', '2018-01-03 9:34:55', '2018-01-07 8:45:43']
['B1234-34', 'B1234-C456', '34', '2018-02-03 10:50:55', '2018-03-04 1:50:32']


In [41]:
#Patient ID- visit mapping
patHashMap

# ```
# {'A1234-B456': ['A1234-12', 'A1234-34', 'A1234-15'],
#  'B1234-C456': ['B1234-13', 'B1234-34']}
# ```

{'A1234-B456': ['A1234-12', 'A1234-34', 'A1234-15'],
 'B1234-C456': ['B1234-13', 'B1234-34']}

In [42]:
# Patient Admission ID- visit date mapping
visitMap

# ```
# {'A1234-12': [datetime.datetime(2019, 1, 3, 0, 0)],
#  'A1234-34': [datetime.datetime(2019, 2, 3, 0, 0)],
#  'A1234-15': [datetime.datetime(2019, 4, 3, 0, 0)],
#  'B1234-13': [datetime.datetime(2018, 1, 3, 0, 0)],
#  'B1234-34': [datetime.datetime(2018, 2, 3, 0, 0)]}
# ```

{'A1234-12': [datetime.datetime(2019, 1, 3, 9, 34, 55)],
 'A1234-34': [datetime.datetime(2019, 2, 3, 10, 50, 55)],
 'A1234-15': [datetime.datetime(2019, 4, 3, 12, 34, 55)],
 'B1234-13': [datetime.datetime(2018, 1, 3, 9, 34, 55)],
 'B1234-34': [datetime.datetime(2018, 2, 3, 10, 50, 55)]}

In [43]:
print('Creating Diagnosis-Visit mapping')
visitDxMap = dict(defaultdict(list))

data = open('data/Diagnosis_Table.csv', 'r')
data.readline()[1:]

for line in data:
    feature = line.strip().split(',')
    visitDxMap.setdefault(feature[0], []).append('D_' + feature[4].split('.')[0]) # add a unique identifier before the
    
visitDxMap # Mapping of each Admission ID to each diagnosis code assigned during that visit

# ```
# {'A1234-12': ['D_E11', 'D_I25', 'D_I25'],
#  'A1234-34': ['D_E11', 'D_I25', 'D_I25', 'D_780', 'D_784'],
#  'A1234-15': ['D_E11', 'D_I25', 'D_I25', 'D_786', 'D_401', 'D_789'],
#  'B1234-13': ['D_M05', 'D_Z13', 'D_O99'],
#  'B1234-34': ['D_M05', 'D_Z13', 'D_O99', 'D_D37']}
# ```

Creating Diagnosis-Visit mapping


{'A1234-12': ['D_E11', 'D_I25', 'D_I25'],
 'A1234-34': ['D_E11', 'D_I25', 'D_I25', 'D_780', 'D_784'],
 'A1234-15': ['D_E11', 'D_I25', 'D_I25', 'D_786', 'D_401', 'D_789'],
 'B1234-13': ['D_M05', 'D_Z13', 'D_O99'],
 'B1234-34': ['D_M05', 'D_Z13', 'D_O99', 'D_D37']}

In [44]:
print("Sorting visit mapping")
patDxVisitOrderMap = {}
for patid, visitDates in patHashMap.items():
    sorted_list = ([(visitMap[visitDateID], visitDxMap[visitDateID]) for visitDateID in visitDates])
    patDxVisitOrderMap[patid] = sorted_list 
  
patDxVisitOrderMap

# ```
# {'A1234-B456': [([datetime.datetime(2019, 1, 3, 0, 0)],
#    ['D_E11', 'D_I25', 'D_I25']),
#   ([datetime.datetime(2019, 2, 3, 0, 0)],
#    ['D_E11', 'D_I25', 'D_I25', 'D_780', 'D_784']),
#   ([datetime.datetime(2019, 4, 3, 0, 0)],
#    ['D_E11', 'D_I25', 'D_I25', 'D_786', 'D_401', 'D_789'])],
#  'B1234-C456': [([datetime.datetime(2018, 1, 3, 0, 0)],
#    ['D_M05', 'D_Z13', 'D_O99']),
#   ([datetime.datetime(2018, 2, 3, 0, 0)],
#    ['D_M05', 'D_Z13', 'D_O99', 'D_D37'])]}
# ```

Sorting visit mapping


{'A1234-B456': [([datetime.datetime(2019, 1, 3, 9, 34, 55)],
   ['D_E11', 'D_I25', 'D_I25']),
  ([datetime.datetime(2019, 2, 3, 10, 50, 55)],
   ['D_E11', 'D_I25', 'D_I25', 'D_780', 'D_784']),
  ([datetime.datetime(2019, 4, 3, 12, 34, 55)],
   ['D_E11', 'D_I25', 'D_I25', 'D_786', 'D_401', 'D_789'])],
 'B1234-C456': [([datetime.datetime(2018, 1, 3, 9, 34, 55)],
   ['D_M05', 'D_Z13', 'D_O99']),
  ([datetime.datetime(2018, 2, 3, 10, 50, 55)],
   ['D_M05', 'D_Z13', 'D_O99', 'D_D37'])]}

In [45]:
print("Extracting patient IDs, visit dates and diagnosis codes into individual lists for encoding")
patIDs = [patid for patid, visitDate in patDxVisitOrderMap.items()]
datesList = [[visit[0][0] for visit in visitDate] for patid, visitDate in patDxVisitOrderMap.items()]
DxsCodesList = [[visit[1] for visit in visitDate] for patid, visitDate in patDxVisitOrderMap.items()]

Extracting patient IDs, visit dates and diagnosis codes into individual lists for encoding


In [46]:
patIDs

# ```
# ['A1234-B456', 'B1234-C456']
# ```

['A1234-B456', 'B1234-C456']

In [47]:
datesList

# ```
# [[datetime.datetime(2019, 1, 3, 0, 0),
#   datetime.datetime(2019, 2, 3, 0, 0),
#   datetime.datetime(2019, 4, 3, 0, 0)],
#  [datetime.datetime(2018, 1, 3, 0, 0), datetime.datetime(2018, 2, 3, 0, 0)]]
# ```

[[datetime.datetime(2019, 1, 3, 9, 34, 55),
  datetime.datetime(2019, 2, 3, 10, 50, 55),
  datetime.datetime(2019, 4, 3, 12, 34, 55)],
 [datetime.datetime(2018, 1, 3, 9, 34, 55),
  datetime.datetime(2018, 2, 3, 10, 50, 55)]]

In [48]:
DxsCodesList

# ```
# [[['D_E11', 'D_I25', 'D_I25'],
#   ['D_E11', 'D_I25', 'D_I25', 'D_780', 'D_784'],
#   ['D_E11', 'D_I25', 'D_I25', 'D_786', 'D_401', 'D_789']],
#  [['D_M05', 'D_Z13', 'D_O99'], ['D_M05', 'D_Z13', 'D_O99', 'D_D37']]]
# ```

[[['D_E11', 'D_I25', 'D_I25'],
  ['D_E11', 'D_I25', 'D_I25', 'D_780', 'D_784'],
  ['D_E11', 'D_I25', 'D_I25', 'D_786', 'D_401', 'D_789']],
 [['D_M05', 'D_Z13', 'D_O99'], ['D_M05', 'D_Z13', 'D_O99', 'D_D37']]]

In [50]:
('Encoding string Dx codes to integers and mapping the encoded integer value to the ICD-10 code for interpretation')
DxCodeDictionary = {}
encodedDxs = []
for patient in DxsCodesList:
    encodedPatientDxs = []
    for visit in patient:
        encodedVisit = []
        for code in visit:
            if code in DxCodeDictionary:
                encodedVisit.append(DxCodeDictionary[code])
            else:
                DxCodeDictionary[code] = len(DxCodeDictionary)
                encodedVisit.append(DxCodeDictionary[code])
        encodedPatientDxs.append(encodedVisit)
    encodedDxs.append(encodedPatientDxs)

In [51]:
DxCodeDictionary # Dictionary of all unique codes in the entire dataset aka: Our Code Vocabulary

# ```
# {'D_E11': 0,
#  'D_I25': 1,
#  'D_780': 2,
#  'D_784': 3,
#  'D_786': 4,
#  'D_401': 5,
#  'D_789': 6,
#  'D_M05': 7,
#  'D_Z13': 8,
#  'D_O99': 9,
#  'D_D37': 10}
# ```

{'D_E11': 0,
 'D_I25': 1,
 'D_780': 2,
 'D_784': 3,
 'D_786': 4,
 'D_401': 5,
 'D_789': 6,
 'D_M05': 7,
 'D_Z13': 8,
 'D_O99': 9,
 'D_D37': 10}

In [52]:
encodedDxs # Converted list of list with integer converted diagnosis codes

# ```
# [[[0, 1, 1], [0, 1, 1, 2, 3], [0, 1, 1, 4, 5, 6]], [[7, 8, 9], [7, 8, 9, 10]]]
# ```

[[[0, 1, 1], [0, 1, 1, 2, 3], [0, 1, 1, 4, 5, 6]], [[7, 8, 9], [7, 8, 9, 10]]]

In [53]:
outFile = 'ArtificialEHR_Data'
print('Dumping files into a pickled list')
pickle.dump(patIDs, open(outFile+'.patIDs', 'wb'),-1)
pickle.dump(datesList, open(outFile+'.dates', 'wb'),-1)
pickle.dump(encodedDxs, open(outFile+'.encodedDxs', 'wb'),-1)
pickle.dump(DxCodeDictionary, open(outFile+'.Dxdictionary', 'wb'),-1)

Dumping files into a pickled list


# Gated Recurrent Unit (GRU)

In [55]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
import numpy as np
import itertools
import pickle
import sys, random
np.random.seed(0)
torch.manual_seed(0)

# check if GPU is available
if(torch.cuda.is_available()):
    print('Training on GPU!')
else: 
    print('Training on CPU!')
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Training on CPU!


In [67]:
def load_data(sequences, labels):
    dataSize = len(labels)
    idx = np.random.permutation(dataSize)
    nTest = int(np.ceil(0.15 * dataSize))
    nValid = int(np.ceil(0.10 * dataSize))

    test_idx = idx[:nTest]
    valid_idx = idx[nTest:nTest+nValid]
    train_idx = idx[nTest+nValid:]

    train_x = sequences[train_idx]
    train_y = labels[train_idx]
    test_x = sequences[test_idx]
    test_y = labels[test_idx]
    valid_x = sequences[valid_idx]
    valid_y = labels[valid_idx]

    train_x = [sorted(seq) for seq in train_x]
    train_y = [sorted(seq) for seq in train_y]
    valid_x = [sorted(seq) for seq in valid_x]
    valid_y = [sorted(seq) for seq in valid_y]
    test_x = [sorted(seq) for seq in test_x]
    test_y = [sorted(seq) for seq in test_y]

    train = (train_x, train_y)
    test = (test_x, test_y)
    valid = (valid_x, valid_y)
    return (train, test, valid)

In [68]:
def padding(seqs, labels, vocab, n_classes):
    lengths = np.array([len(seq) for seq in seqs]) - 1 # remove the last list in each patient's sequences for labels
    n_samples = len(lengths)
    maxlen = np.max(lengths)

    x = torch.zeros(maxlen, n_samples, vocab) # maxlen = number of visits, n_samples = samples
    y = torch.zeros(maxlen, n_samples, n_classes)
    mask = torch.zeros(maxlen, n_samples)
    for idx, (seq,label) in enumerate(zip(seqs,labels)):
        for xvec, subseq in zip(x[:,idx,:], seq[:-1]):
            xvec[subseq] = 1.
        for yvec, subseq in zip(y[:,idx,:], label[1:]):
            yvec[subseq] = 1.
        mask[:lengths[idx], idx] = 1.
        
    return x, y, lengths, mask

In [69]:
### Dataset generator:
class Dataset():
    def __init__(self, x, y):
        self.x, self.y = x, y
    def __len__(self): return len(self.x)
    def __getitem__(self, i ): return self.x[i], self.y[i]

In [70]:
### Random Sampler class:
class Sampler():
    def __init__(self, ds, bs, shuffle=False):
        self.n,self.bs,self.shuffle = (len(ds)//bs)*bs,bs,shuffle  
        # Note: self.n = (len(ds)//bs) keeps the exact amount of samples needed for your desired batchSize
        
    def __iter__(self):
        self.idxs = torch.randperm(self.n) if self.shuffle else torch.arange(self.n)
        for i in range(0, self.n, self.bs): yield self.idxs[i:i+self.bs]

In [71]:
### DataLoader:
def collate(batch_pairs):
    x,y = zip(*batch_pairs)
    return (x,y)

In [72]:
class DataLoader():
    def __init__(self, ds, sampler, collate_fn=collate):
        self.ds,self.sampler,self.collate_fn = ds,sampler,collate_fn
    def __len__(self): return len(self.ds)
    def __iter__(self):
        for s in self.sampler: yield self.collate_fn([self.ds[i] for i in s])

In [73]:
### Embedding Layer:
class Custom_Embedding(nn.Module):
    def __init__ (self, inputDimSize, embSize):
        super(Custom_Embedding, self).__init__()
        self.inputDimSize = inputDimSize
        self.embSize = embSize
        
        self.W_emb = nn.Parameter(torch.randn(self.inputDimSize, self.embSize) * 0.01)
        self.b_emb = nn.Parameter(torch.zeros(self.embSize) * 0.01) 
       
    def forward(self, x):
        return torch.tanh(x@self.W_emb + self.b_emb)

In [74]:
### Drop out Layer:
def dropout_mask(x, sz, p):
    return x.new(*sz).bernoulli_(1-p).div_(1-p)

In [75]:
### GRU Cell:
class EHR_GRU(Custom_Embedding):
    def __init__(self, inputDimSize, hiddenDimSize, embSize, numClass, numLayers):
        super().__init__(inputDimSize, embSize)
        
        self.numClass = numClass
        self.numLayers = numLayers
        self.hiddenDimSize = hiddenDimSize
        self.emb = Custom_Embedding(inputDimSize, embSize)
        
        self.W_r = nn.Parameter(torch.randn(embSize, hiddenDimSize)* 0.01)
        self.W_z = nn.Parameter(torch.randn(embSize, hiddenDimSize)* 0.01)
        self.W_h = nn.Parameter(torch.randn(embSize, hiddenDimSize)* 0.01)
        
        self.U_r = nn.Parameter(torch.randn(hiddenDimSize, hiddenDimSize)* 0.01)
        self.U_z = nn.Parameter(torch.randn(hiddenDimSize, hiddenDimSize)* 0.01)
        self.U_h = nn.Parameter(torch.randn(hiddenDimSize, hiddenDimSize)* 0.01)
        
        self.b_r = nn.Parameter(torch.randn(hiddenDimSize))
        self.b_z = nn.Parameter(torch.randn(hiddenDimSize))
        self.b_h = nn.Parameter(torch.randn(hiddenDimSize))
        
        self.W_output = nn.Parameter(torch.randn(embSize, numClass))
        self.b_output = nn.Parameter(torch.randn(numClass))
        
    def forward(self, emb, mask):
        h = self.init_hidden(emb.size(1))
        
        z = torch.sigmoid(emb@self.W_z + h@self.U_z + self.b_z) 
        r = torch.sigmoid(emb@self.W_r + h@self.U_r + self.b_r)
        h_tilde = torch.tanh(emb@self.W_h + (r * h)@self.U_h + self.b_h)
        h_new = z * h + ((1. - z) * h_tilde)
        h_new = mask[:, :, None] * h_new + (1. - mask)[:, :, None] * h
       
        return h_new
    
    def init_hidden(self, batchSize):
        return Variable(torch.zeros(1, batchSize, hiddenDimSize))

In [76]:
### GRU Layer:
class build_EHR_GRU(EHR_GRU):
    def __init__(self, GRUCell, *kwargs):
        super().__init__(inputDimSize, hiddenDimSize, embSize, numClass, numLayers)
        self.cell = GRUCell(*kwargs)
        self.emb = Custom_Embedding(inputDimSize, embSize)

    def forward(self, x, mask):
        inputVector = self.emb(x)
        for i in range(numLayers):
            memories = self.cell(inputVector, mask)
            drop_out = dropout_mask(inputVector, (inputVector.size(0), 1, inputVector.size(2)), 0.5)
            inputVector = memories * drop_out
        
        y_linear = inputVector@self.W_output + self.b_output
        output = F.softmax(y_linear, dim=1)
        output = output * mask[:,:,None]
        return output, inputVector

In [77]:
### Loss Function:
class cost_function():
    def __init__(self, yhat, y, L_2=0.001, logEps=1e-8):
        self.yhat = yhat
        self.y = y
       
        self.logEps = logEps
        self.L_2 = L_2
        
        self.W_out = nn.Parameter(torch.randn(hiddenDimSize, numClass)*0.01)
        
    def cross_entropy(self):
        return  -(self.y * torch.log(self.yhat + self.logEps) + (1. - self.y) * torch.log(1. - self.yhat + self.logEps))
    
    def prediction_loss(self):
        return  (torch.sum(torch.sum(self.cross_entropy(), dim=0),dim=1)).float()/  lengths.float()
    
    def cost(self):
        return torch.mean(self.prediction_loss()) + self.L_2 * (self.W_out ** 2).sum() # regularize
    

In [78]:
### Training Loop:
optimizer = torch.optim.Adadelta(model.parameters(), lr = 0.01, rho=0.95)
epochs = 10

counter = 0
for e in range(epochs):
    for x, y in train_dl:
        x, y , mask, lengths = padding(x, y, inputDimSize, numClass)
        
        output, h = model(x, mask)
        
        loss = cost_function(output, y).cost()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        optimizer.zero_grad()
    
    with torch.no_grad():
            model.eval()
            val_loss = []
            for x_valid, y_valid in valid_dl:
                    x_val, y_val, mask, lengths = padding(x_valid, y_valid, inputDimSize, numClass)
                    outputs_val, hidden_val = model(x_val,  mask)
                    loss = cost_function(outputs_val, y_val).cost()
                    val_loss.append(loss.item())
            model.train()

            print("Epoch: {}/{}...".format(e+1, epochs),
                                  "Step: {}...".format(counter),
                                  "Training Loss: {:.4f}...".format(loss.item()),
                                  "Val Loss: {:.4f}".format(torch.mean(torch.tensor(val_loss))))   


NameError: name 'model' is not defined